In [1]:
import pandas as pd
import os
import requests
import sys
import io

def getGoogleSheet(spreadsheet_id, gid, outDir, outFile):
    url = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={gid}'
    response = requests.get(url)
    if response.status_code == 200:
        filepath = os.path.join(outDir, outFile)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f'CSV file saved to: {filepath}')
        return pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    else:
        print(f'Error downloading Google Sheet: {response.status_code}')
        sys.exit(1)

def get_sheetdata():
    spreadsheet_id = '1mhwOLqPu2F9026EQiVxFPIN1t9RGafGpl-dokaIsm9c'
    
    # Replace these GIDs with the actual GIDs of your sheets
    darko_gid = '142925152' # Assuming 'darko' is the first sheet
    timedecay_gid = '923517192'  # Replace with the actual GID of 'timedecay' sheet
    
    darko = getGoogleSheet(spreadsheet_id, darko_gid, '', 'darko.csv')
    time_decay = getGoogleSheet(spreadsheet_id, timedecay_gid, '', 'timedecay.csv')
    if 'date' in time_decay.columns:
        time_decay.drop(columns='date',inplace=True)
    
    return darko, time_decay

darko,time_decay=get_sheetdata()


CSV file saved to: darko.csv
CSV file saved to: timedecay.csv


In [2]:


dir = ''

lebron_file = dir + 'lebron.csv'
epm_file = dir + 'epm.csv'
darko_file = dir + 'darko.csv'
time_decay_file = dir + 'timedecay.csv'
long_rapm_file = dir + '28year.csv'
recency_rapm_file = dir + 'recency_RAPM.csv'

lebron_df = pd.read_csv(lebron_file)
epm_df = pd.read_csv(epm_file)
darko_df = pd.read_csv(darko_file)
time_decay_df = pd.read_csv(time_decay_file)
long_rapm_df = pd.read_csv(long_rapm_file)
recency_rapm_df = pd.read_csv(recency_rapm_file)
lebron_df.rename(columns={'NBA ID':'nba_id'},inplace=True)
recency_rapm_df.rename(columns={'Defense':'RecentD', 'Offense':'RecentO', 'Total':'RecentT'},inplace=True)

lebron_df = lebron_df[lebron_df.year==2024]

In [3]:
lebron_df['orapm_rank_lebron']=lebron_df['O-LEBRON'].rank()
lebron_df['drapm_rank_lebron']=lebron_df['D-LEBRON'].rank()

lebron_df['rapm_rank_lebron']=lebron_df['LEBRON'].rank()

darko_df['orapm_rank_darko'] = darko_df['o_dpm'].rank(ascending=False)
darko_df['drapm_rank_darko'] = darko_df['d_dpm'].rank(ascending=False)
darko_df['rapm_rank_darko'] = darko_df['dpm'].rank(ascending=False)

# Create ranking columns for Time Decay RAPM dataframe
time_decay_df['orapm_rank_timedecay'] = time_decay_df['orapm'].rank(ascending=False)
time_decay_df['drapm_rank_timedecay'] = time_decay_df['drapm'].rank(ascending=False)
time_decay_df['rapm_rank_timedecay'] = time_decay_df['rapm'].rank(ascending=False)
dataframes = [epm_df, darko_df, time_decay_df, long_rapm_df,recency_rapm_df]
df = lebron_df
print(df.columns)
for frame in dataframes:

    if 'player_name' in frame.columns:
        frame=frame.drop(columns='player_name')
    frame['nba_id'] = frame['nba_id'].astype(int)
    df = df.merge(frame)
    df['nba_id']=df['nba_id'].astype(int)


column_rename_dict = {
    # LEBRON metrics
    'O-LEBRON': 'orapm_lebron',
    'D-LEBRON': 'drapm_lebron',
    'LEBRON': 'rapm_lebron',
    # EPM metrics
    'oepm': 'orapm_epm',
    'depm': 'drapm_epm',
    'epm': 'rapm_epm',
    'OEPM_Rank': 'orapm_rank_epm',
    'DEPM_Rank': 'drapm_rank_epm',
    'EPM_Rank': 'rapm_rank_epm',
    # DARKO metrics
    'o_dpm': 'orapm_darko',
    'd_dpm': 'drapm_darko',
    'dpm': 'rapm_darko',
    # Time Decay RAPM metrics
    'orapm': 'orapm_timedecay',
    'drapm': 'drapm_timedecay',
    'rapm': 'rapm_timedecay',
    # FiveThirtyEight (Long-term) metrics
    'Offense': 'orapm_long',
    'Defense': 'drapm_long',
    'Total': 'rapm_long',
    'Offense_Rank': 'orapm_rank_long',
    'Defense_Rank': 'drapm_rank_long',
    'Total_Rank': 'rapm_rank_long',
    # Recent metrics
    'RecentO': 'orapm_recent',
    'RecentD': 'drapm_recent',
    'RecentT': 'rapm_recent',
    'off rk': 'orapm_rank_recent',
    'def rk': 'drapm_rank_recent',
    'tot rk': 'rapm_rank_recent'
}


df.rename(columns = column_rename_dict,inplace=True)
rank_col =[col for col in df.columns if '_rank'  in col]
rapm_col =[col for col in df.columns if 'rapm_'  in col]
keepcol=list(set(rank_col+rapm_col))
keepcol = ['nba_id','name','Minutes','Games','Age']+keepcol
df=df[keepcol]
df.to_csv('comp.csv',index = False)

Index(['bref_id', 'Player', 'Season', 'team', 'Offensive Archetype', 'Minutes',
       'Games', 'Pos', 'Age', 'LEBRON', 'O-LEBRON', 'D-LEBRON', 'WAR',
       'Value Added', 'nba_id', 'boxLEBRON', 'boxOLEBRON', 'boxDLEBRON',
       'year', 'Defensive Role', 'orapm_rank_lebron', 'drapm_rank_lebron',
       'rapm_rank_lebron'],
      dtype='object')


In [4]:
df


,nba_id,name,Minutes,Games,Age,orapm_rank_long,orapm_rank_recent,orapm_lebron,rapm_lebron,rapm_rank_epm,...,rapm_long,orapm_epm,drapm_rank_long,orapm_long,drapm_recent,drapm_long,drapm_rank_epm,drapm_darko,rapm_darko,orapm_darko
0,1630639,A.J. Lawson,251.766667,35.0,23.0,1517,458,-0.233967,-0.822179,434,...,-5.1,-3.1,1567,-3.3,-0.9,-1.8,373,-1.017110,-2.171622,-1.154511
1,1631260,AJ Green,495.916667,48.0,24.0,1031,247,-0.258833,-1.253518,261,...,-4.3,-0.5,2152,-1.9,-0.9,-2.4,386,-0.897139,-1.742007,-0.844868
2,1631100,AJ Griffin,132.500000,18.0,20.0,363,184,-1.425417,-1.462522,515,...,-0.5,-5.8,976,0.3,-0.4,-0.8,504,-1.023133,-2.024035,-1.000902
3,203932,Aaron Gordon,2085.633333,66.0,28.0,139,78,0.580835,1.145196,49,...,2.9,0.4,258,1.8,1.5,1.1,28,1.101658,1.871826,0.770168
4,1628988,Aaron Holiday,1135.333333,69.0,27.0,412,223,-1.041948,-1.294685,240,...,1.3,-1.2,234,0.1,1.5,1.2,207,-0.148524,-0.781600,-0.633076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,1628380,Zach Collins,1355.133333,60.0,26.0,329,401,-1.618539,-1.766281,314,...,-0.2,-1.4,884,0.4,-0.1,-0.6,359,0.136355,-1.735236,-1.871591
388,203897,Zach LaVine,871.850000,25.0,28.0,312,207,0.245138,-0.187196,125,...,-2.5,0.1,2547,0.5,0.3,-3.0,149,-0.820504,0.072880,0.893384
389,1630192,Zeke Nnaji,521.600000,53.0,23.0,1383,478,-2.090400,-1.744069,427,...,-3.0,-3.4,637,-2.9,-0.0,-0.1,276,-0.273936,-2.971895,-2.697960
390,1630533,Ziaire Williams,1037.666667,51.0,22.0,972,430,-1.783639,-2.284760,354,...,-2.6,-3.5,1031,-1.7,-0.9,-0.9,152,-0.532994,-1.928329,-1.395334
